<a href="https://colab.research.google.com/github/gmmanzo/Seizure-Detection/blob/main/Helper_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def extract_features(windows):
    features = {}

    # Time-domain features
    features['mean'] = np.mean(windows)
    features['std'] = np.std(windows)
    features['skew'] = skew(windows)
    features['kurtosis'] = kurtosis(windows)

    # Frequency-domain features (Power Spectral Density)
    f, Pxx = welch(windows, fs=512, nperseg=512)
    features['power_band'] = np.sum(Pxx)  # Total power

    return features